In [1]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np

In [2]:
from ipywidgets import interact, ToggleButtons, interactive, HBox, VBox, Label
from ipywidgets import AppLayout, FloatSlider

plt.ioff()

plt.rcParams.update({'font.size': 12})

#core half thinkness in microns
gw_slider = FloatSlider(description='Guide Width', min=0.4, max=1.0, step=0.1, value=1.0);

#vacuum wavelength in microns
wavelength=1.55
#refractive index of core
n1=3.5
#refractive inex of cladding
n0=3.2

#Finding the 1st Even Mode

def modecalc(d):
    a=d/2
    V=(2*np.pi*a/wavelength)*(n1**2.0-n0**2.0)**0.5
    Ka_arr = np.arange(80,100)/100.0
    Diff_arr = (n0**2/n1**2)*Ka_arr*np.tan(Ka_arr)-(V**2-Ka_arr**2)**0.5
    Diff_arr=np.absolute(Diff_arr)
    Ka0 = np.argmin(Diff_arr)/100.0+1.0
    Ga0 = (n0**2/n1**2)*Ka_arr[np.argmin(Diff_arr)]*np.tan(Ka_arr[np.argmin(Diff_arr)])
    Beta0 = ((n1*(2*np.pi/wavelength))**2-(Ka0/a)**2)**0.5
    neff = Beta0*wavelength/2/np.pi
    x0=np.linspace(-2,-a,151)
    y0=(np.cos(Ka0)*np.exp((a+x0)*(Ga0/a)))**2
    x1=np.linspace(-a,a,151)
    y1=np.cos(Ka0*x1/a)**2
    x2=np.linspace(a,2,151)
    y2=(np.cos(Ka0)*np.exp((a-x2)*(Ga0/a)))**2
    conf = (np.sum((np.cos(Ka0*x1/a)**2))*(2*a)/(np.sum(np.cos(Ka0*x1/a)**2)*(2*a)
            + 2* np.sum((np.cos(Ka0)*np.exp((a-x2)*(Ga0/a)))**2)*(2-a)))
    return a,x0,y0,x1,y1,x2,y2,conf,neff
a,x0,y0,x1,y1,x2,y2,conf,neff = modecalc(1.0)

fig2=plt.figure(figsize=(5,4))
fig2.canvas.layout.width = '600px'
fig2.canvas.layout.height = '500px'
# fig2.canvas.header_visible = False
lines0 = plt.plot(x0, y0, color='blue') 
lines1 = plt.plot(x1, y1, color='blue') 
lines2 = plt.plot(x2, y2, color='blue')
plt.xlabel('Position (micron)')
plt.ylabel('Intensity')
vline1 = plt.plot([a,a],[0,1.0], color='red')
vline2 = plt.plot([-a,-a],[0,1.0], color='red')

plt.text(-2.0,0.8,'ncore = '+ str(np.round(n0,1)))
plt.text(-2.0,0.7,'nclad = '+ str(np.round(n1,1)))
plt.text(0.6,0.8,'neff = '+ str(round(neff,2)))
plt.text(0.6,0.7,'Conf Fact = '+ str(np.round(conf,2)))
nave = n1*conf + n0*(1-conf)
plt.text(0.6,0.6,'nave = '+ str(np.round(nave,2)))

def update(change):
    a,x0,y0,x1,y1,x2,y2,conf,neff = modecalc(change.new)
    a=change.new/2
    lines0[0].set_data(x0,y0)
    lines1[0].set_data(x1,y1)
    lines2[0].set_data(x2,y2)
    vline1[0].set_data([a,a],[0,1.0])
    vline2[0].set_data([-a,-a],[0,1.0])   
    fig2.canvas.draw()
    fig2.canvas.flush_events()    

gw_slider.observe(update, names='value')

AppLayout(
    header=gw_slider,
    left_sidebar=None,
    center=fig2.canvas,
    right_sidebar=None,
    pane_heights=[0, 6, 5],
    pane_widths=[0,6,3])

AppLayout(children=(FloatSlider(value=1.0, description='Guide Width', layout=Layout(grid_area='header'), max=1…